In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# ---------- SETTINGS ----------
start_date = datetime(2025, 1, 6)
days_open = 24   # 4 weeks Mon–Sat

price_map = {
    "Chicken": 50,
    "Pork": 60,
    "Fish": 40,
    "Vegetable": 40,
    "Beef": 80,
    "Soup": 60
}

# menu pools
chicken = ["Chicken Adobo", "Chicken Curry", "Kalderetang Manok", "Chicken BBQ"]
pork = ["Pork Menudo", "Pork Adobo", "Sisig", "Dinakdakan"]
fish = ["Fried Tilapia", "Ginataang Tulingan", "Paksiw na Bangus","Fried Salmon", "Ginataang Pagi",]
vegetable = ["Chopsuey", "Laing", "Ginataang Kalabasa", "Labong", "Monggo"]
beef = ["Beef Steak"]
soup = ["Sinigang na Baboy", "Nilagang Baka", "Tinolang Manok", "Nilagang Pata", "Yellow Fin Tuna Soup"]


rare = ["Ginataang Pagi", "Dinakdakan", "Sisig", "Beef Steak", "Yellow Fin Tuna Soup", "Nilagang Baka", "Nilagang Pata"]

base_cost = {
    "Chicken": 650,
    "Pork": 720,
    "Fish": 300,
    "Vegetable": 250,
    "Beef": 820,
    "Soup": 600
}

# ---------- HELPERS ----------
def cost_fluctuate(base):
    return round(base * random.uniform(0.9, 1.1))

def payday(d):
    return d.day in [15, 30]

# ---------- GENERATION ----------
rows = []
date = start_date
count = 0

while count < days_open:

    if date.weekday() == 6:
        date += timedelta(days=1)
        continue

    menu = []

    menu.append(("Soup", random.choice(soup)))
    menu.append(("Chicken", random.choice(chicken)))
    menu.append(("Pork", random.choice(pork)))
    menu.append(("Pork", random.choice(pork)))
    menu.append(("Fish", random.choice(fish)))
    menu.append(("Fish", random.choice(fish)))
    menu.append(("Vegetable", random.choice(vegetable)))

    if random.random() < 0.4:
        menu.append(("Beef", random.choice(beef)))

    has_pork = any(x[0]=="Pork" for x in menu)
    has_beef = any(x[0]=="Beef" for x in menu)

    for dtype, name in menu:

        if dtype == "Soup":
            total = random.randint(10,12)
            sold = total
            unsold = 0
        else:
            total = random.randint(12,15)

            if name in rare:
                unsold = random.randint(0,1)
            else:
                unsold = random.randint(0,2)

            if payday(date):
                unsold = max(0, unsold-2)

            if dtype=="Chicken" and has_pork and has_beef:
                unsold = max(3, unsold)

            sold = total - unsold

        cost = cost_fluctuate(base_cost[dtype])
        price = price_map[dtype]
        revenue = sold * price
        profit = revenue - cost

        rows.append([
            date.strftime("%Y-%m-%d"),
            date.strftime("%A"),
            name,
            dtype,
            cost,
            price,
            total,
            sold,
            unsold,
            revenue,
            profit
        ])

    date += timedelta(days=1)
    count += 1

cols = [
    "Date","Day","Dish","Dish_Type","Cost_PHP","Price_PHP",
    "Total_Serving","Sold","Unsold","Revenue","Profit"
]

df = pd.DataFrame(rows, columns=cols)
df.to_excel("filipino_eatery_realistic_month.xlsx", index=False)

print("DATASET CREATED")

DATASET CREATED


In [2]:
import pandas as pd
import random

# ---------- LOAD DATA ----------
synthetic = pd.read_excel("filipino_eatery_realistic_month.xlsx")
real = pd.read_csv("real_menu.csv")

# ---------- CLEAN STRINGS ----------
def clean_text(s):
    return str(s).strip().lower()

synthetic["Dish_clean"] = synthetic["Dish"].apply(clean_text)
real["Dish_clean"] = real["Dish"].apply(clean_text)

# ---------- BUILD LOOKUP ----------
cost_lookup = dict(zip(real["Dish_clean"], real["Cost_PHP"]))
price_lookup = dict(zip(real["Dish_clean"], real["Price_PHP"]))

# ---------- COST FLUCTUATION FUNCTION ----------
def fluctuate_cost(base_cost):
    return round(base_cost * random.uniform(0.9, 1.1))

# ---------- UPDATE SYNTHETIC ----------
for i, row in synthetic.iterrows():
    key = row["Dish_clean"]

    if key in cost_lookup:

        # apply ±10% fluctuation
        fluctuated_cost = fluctuate_cost(cost_lookup[key])

        synthetic.at[i, "Cost_PHP"] = fluctuated_cost
        synthetic.at[i, "Price_PHP"] = price_lookup[key]

# ---------- RECOMPUTE ECONOMICS ----------
synthetic["Revenue"] = synthetic["Sold"] * synthetic["Price_PHP"]
synthetic["Profit"] = synthetic["Revenue"] - synthetic["Cost_PHP"]

# ---------- CLEANUP ----------
synthetic.drop(columns=["Dish_clean"], inplace=True)

# ---------- SAVE ----------
synthetic.to_excel("final_realism_upgraded_dataset.xlsx", index=False)

print("✅ Real-world costs applied with ±10% fluctuation!")


✅ Real-world costs applied with ±10% fluctuation!
